# TFIDF Regressions

### Functions in use

In [1]:
# takes df and extracts labels for each companies
def return_label(df,end_year):
    return_dict={}
    companies=pd.DataFrame(df[str(end_year)].dropna())
    
    for cik, label in companies.iterrows():
        return_dict[cik]=label[str(end_year)]
        
    return return_dict

In [2]:
# extracts regressors and target
from tqdm import tqdm
def get_xy(corpus,labels):
    X=[]
    Y=[]
    for ix in tqdm(labels.keys()):
        if corpus.get(str(ix),""):
            X.append(corpus[str(ix)])
            Y.append(labels[ix])
    return X,Y

In [3]:
# function for training models
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, precision_score, recall_score
import numpy as np

def do_train_10f(corpus, year):
    labels = return_label(rev_df, year)
    X, Y = get_xy(corpus, labels)
    
    # Train tf-idf vectors on the train-set
    vectorizer = TfidfVectorizer(stop_words="english")
    X = [" ".join(i) for i in X]
    X = vectorizer.fit_transform(X)  # X remains a sparse matrix
    Y = np.array(Y)

    # Initialize lists to store performance metrics for each fold
    lr_train_accs, lr_test_accs, svc_train_accs, svc_test_accs = [], [], [], []
    lr_prs, svc_prs = [], []
    lr_recals, svc_recals = [], []
    lr_f1s, svc_f1s = [], []
    ovr_aurocs, ovo_aurocs = [], []

    # Create a 10-fold stratified cross-validation splitter
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    for train_indices, test_indices in skf.split(X, Y):
        x_train, x_test = X[train_indices], X[test_indices]
        y_train, y_test = Y[train_indices], Y[test_indices]

        # Train-test split within the fold (0.1 test size)
        x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.1, stratify=y_train)

        # Logistic Regression
        lr = LogisticRegression(n_jobs=-1, max_iter=1000)
        lr.fit(x_train, y_train)

        lr_train_acc = lr.score(x_train, y_train)
        lr_test_acc = lr.score(x_test, y_test)

        lr_pr = precision_score(y_test, lr.predict(x_test), average="weighted", zero_division=0)
        lr_rec = recall_score(y_test, lr.predict(x_test), average="weighted", zero_division=0)
        lr_f1 = precision_recall_fscore_support(y_test, lr.predict(x_test), average="weighted", zero_division=0)

        ovr_auroc = roc_auc_score(y_test, lr.predict_proba(x_test), multi_class="ovr", average="weighted")
        ovo_auroc = roc_auc_score(y_test, lr.predict_proba(x_test), multi_class="ovo", average="weighted")

        # Linear SVC
        l_svc = LinearSVC(dual=False)
        l_svc.fit(x_train, y_train)

        svc_train_acc = l_svc.score(x_train, y_train)
        svc_test_acc = l_svc.score(x_test, y_test)

        svc_pr = precision_score(y_test, l_svc.predict(x_test), average="weighted", zero_division=0)
        svc_rec = recall_score(y_test, l_svc.predict(x_test), average="weighted", zero_division=0)
        svc_f1 = precision_recall_fscore_support(y_test, l_svc.predict(x_test), average="weighted", zero_division=0)

        # Append performance metrics to lists
        lr_train_accs.append(lr_train_acc)
        lr_test_accs.append(lr_test_acc)
        svc_train_accs.append(svc_train_acc)
        svc_test_accs.append(svc_test_acc)
        lr_prs.append(lr_pr)
        lr_recals.append(lr_rec)
        lr_f1s.append(lr_f1)
        ovr_aurocs.append(ovr_auroc)
        ovo_aurocs.append(ovo_auroc)
        svc_prs.append(svc_pr)
        svc_recals.append(svc_rec)
        svc_f1s.append(svc_f1)

    # Calculate and return the mean values of the performance metrics
    return lr_train_accs, lr_test_accs, svc_train_accs, svc_test_accs, lr_prs, lr_recals, lr_f1s, svc_prs, svc_recals, svc_f1s, ovr_aurocs, ovo_aurocs


### Training models

In [ ]:
import pickle
import pandas as pd
main_dict = {}

for year in tqdm(range(2011, 2021)):
    year = str(year)
    # pickle file for k-10 filings
    base_corpus=pickle.load(open(f"TFIDF/future/{year}_future_corpus.pkl","rb"))

    # labels csv file
    rev_df=pd.read_csv(f"labels/{year}.csv",index_col=1)

    performance_dict={'columns': ["lr_train_acc", "lr_test_acc", "svc_train_acc", "svc_test_acc", "lr_precision", "lr_recall", "lr_f1", "svc_precision", "svc_recall", "svc_f1", "OVO_auroc", "OVR_auroc"]}
    for i in range(int(year)+1,2023):
        i = str(i)
        performance_dict[i]=do_train_10f(base_corpus,i)
    main_dict[f"base_{year}"] = performance_dict


100%|██████████| 3650/3650 [00:00<00:00, 846046.40it/s]

100%|██████████| 3673/3673 [00:00<00:00, 360788.73it/s]

100%|██████████| 3638/3638 [00:00<00:00, 869798.66it/s]

100%|██████████| 3664/3664 [00:00<00:00, 871988.76it/s]

100%|██████████| 3682/3682 [00:00<00:00, 859879.03it/s]

100%|██████████| 3675/3675 [00:00<00:00, 812506.84it/s]

100%|██████████| 3653/3653 [00:00<00:00, 729956.77it/s]

100%|██████████| 3701/3701 [00:00<00:00, 820287.42it/s]

100%|██████████| 3610/3610 [00:00<00:00, 794200.76it/s]

100%|██████████| 3681/3681 [00:00<00:00, 681282.90it/s]

100%|██████████| 3865/3865 [00:00<00:00, 817415.54it/s]

100%|██████████| 3799/3799 [00:00<00:00, 734293.13it/s]

100%|██████████| 3831/3831 [00:00<00:00, 792678.14it/s]

100%|██████████| 3870/3870 [00:00<00:00, 871280.54it/s]

100%|██████████| 3857/3857 [00:00<00:00, 742867.73it/s]

100%|██████████| 3842/3842 [00:00<00:00, 774736.34it/s]

100%|██████████| 3898/3898 [00:00<00:00, 741166.73it/s]

100%|██████████| 3794/3794 [00:

### Saving the results into a pickle file

In [ ]:
import pickle

# Create a dictionary
my_dict = main_dict

# Specify the filename for the pickle file
pickle_filename = 'tfidf_future.pkl'

# Open the file in binary write mode and use pickle.dump to save the dictionary
with open(pickle_filename, 'wb') as pickle_file:
    pickle.dump(my_dict, pickle_file)

print(f'Dictionary has been pickled and saved to {pickle_filename}')

### Training future models

In [ ]:
import pickle
import pandas as pd
main_dict = {}

for year in tqdm(range(2011, 2021)):
    year = str(year)
    # pickle file for k-10 filings
    base_corpus=pickle.load(open(f"TFIDF/base/{year}_base_corpus.pkl","rb"))

    # labels csv file
    rev_df=pd.read_csv(f"labels/{year}.csv",index_col=1)

    performance_dict={'columns': ["lr_train_acc", "lr_test_acc", 
                                  "svc_train_acc", "svc_test_acc", 
                                  "lr_precision", "lr_recall", "lr_f1", 
                                  "svc_precision", "svc_recall", "svc_f1", 
                                  "OVO_auroc", "OVR_auroc"]}
    for i in range(int(year)+1,2023):
        i = str(i)
        performance_dict[i]=do_train_10f(base_corpus,i)
    main_dict[f"base_{year}"] = performance_dict


### Saving the results into a pickle file

In [ ]:
import pickle

# Create a dictionary
my_dict = main_dict

# Specify the filename for the pickle file
pickle_filename = 'tfidf_base.pkl'

# Open the file in binary write mode and use pickle.dump to save the dictionary
with open(pickle_filename, 'wb') as pickle_file:
    pickle.dump(my_dict, pickle_file)

print(f'Dictionary has been pickled and saved to {pickle_filename}')